In [1]:
import pandas as pd
import random

# Simulation Project - Newsdealer's Problem

## Define function to run simulation

In [2]:
# newsdealer simulation function
def simulate_newspaper(cost_price, selling_price, scrap_price, order_quantity, ndays, newstype_dist, demand_dist, random_numbers):
        
    # initialize empty dataframe
    df = pd.DataFrame(columns=['Day','RN Newsday Type','Newsday Type','RN Demand','Demand','Revenue','Lost Profit','Scrap Sale','Daily Profit'])
    
    # index to interate over random numbers
    idx = 0
    
    # calculate cost of newspapers
    cost_of_newspapers = order_quantity * cost_price
    
    # repeat the following from day 1 to n number of days
    for i in range(1,ndays+1):
        
        # get newstype rn then newstype
        rn_newstype = random_numbers[idx]
        idx = idx + 1
        newstype = newstype_dist["newsday type"][newstype_dist["cprob"] >= rn_newstype].tolist()[0]
        
        # get demand rn then demand
        rn_demand = random_numbers[idx]
        idx = idx + 1
        demand = demand_dist["demand"][demand_dist[f"{newstype} cprob"] >= rn_demand].tolist()[0]
        
        # calculate revenue, lost profit and scrap
        if (demand <= order_quantity):
            revenue = selling_price*demand
            lost_profit = 0
            scrap = (order_quantity - demand) * scrap_price
        else:
            revenue = selling_price*order_quantity
            lost_profit = (demand - order_quantity) * (selling_price - cost_price)
            scrap = 0
        
        # calculate day profit
        profit = revenue - cost_of_newspapers - lost_profit + scrap
        
        # create new row for this day and add it to dataframe
        new_row = {
            "Day": i,
            "RN Newsday Type": rn_newstype,
            "Newsday Type": newstype,
            "RN Demand": rn_demand,
            "Demand": demand,
            "Revenue": round(revenue * 0.01, 2),
            "Lost Profit": round(lost_profit * 0.01, 2),
            "Scrap Sale": round(scrap * 0.01, 2),
            "Daily Profit": round(profit * 0.01, 2)
        }
        df.loc[i] = new_row
    
    # return the dataframe with all the calculations and the total profit variable
    total_profit = sum(df["Daily Profit"])
    return df, total_profit

## Take Parameters as user input

In [3]:
cost_price = int(input('The price at which the dealer buys the newspapers (in cent): '))

The price at which the dealer buys the newspapers (in cent): 33


In [4]:
selling_price = int(input('The price at which the dealer sells the newspapers (in cent): '))

The price at which the dealer sells the newspapers (in cent): 50


In [5]:
scrap_price = int(input('Scrap price (in cent): '))

Scrap price (in cent): 5


In [6]:
order_quantity = int(input('Order quantity (in bundles of 10): '))

Order quantity (in bundles of 10): 70


In [7]:
ndays = int(input('Number of days to run the simulation: '))

Number of days to run the simulation: 5


In [8]:
newsday_type = ['good','fair','poor']
newsday_prob = []

print('Enter probabilities for Newsday Types good, fair, poor respectively:')
print()
for i in range(0, 3):
        newsday_prob.append(float(input()))

print()

# add probabilities to dataframe and create cumulative probability column
newstype_dist = pd.DataFrame({
    "newsday type": newsday_type,
    "prob": newsday_prob
})

newstype_dist["cprob"] = newstype_dist["prob"].cumsum()

print(newstype_dist)

Enter probabilities for Newsday Types good, fair, poor respectively:

0.35
0.45
0.2

  newsday type  prob  cprob
0         good  0.35   0.35
1         fair  0.45   0.80
2         poor  0.20   1.00


In [9]:
demand_values = [40, 50, 60, 70, 80, 90, 100]
demand_dist = pd.DataFrame({
    "demand": demand_values
})

print('Enter probabilities for demand values from 40 t 100 (7 numbers for each newsday type):')
print()
for type in newsday_type:
    demand_prob = []
    print(f'Demand probailities for {type} newsdays:')
    print()
    for i in range(0, 7):
        demand_prob.append(float(input()))
    print()
    demand_dist[f'{type} prob'] = demand_prob

# add probabilities to dataframe and create cumulative probability columns
demand_dist["good cprob"] = demand_dist["good prob"].cumsum()
demand_dist["fair cprob"] = demand_dist["fair prob"].cumsum()
demand_dist["poor cprob"] = demand_dist["poor prob"].cumsum()

print(demand_dist)

Enter probabilities for demand values from 40 t 100 (7 numbers for each newsday type):

Demand probailities for good newsdays:

0.03
0.05
0.15
0.20
0.35
0.15
0.07

Demand probailities for fair newsdays:

0.10
0.18
0.40
0.20
0.08
0.04
0

Demand probailities for poor newsdays:

0.44
0.22
0.16
0.12
0.06
0
0

   demand  good prob  fair prob  poor prob  good cprob  fair cprob  poor cprob
0      40       0.03       0.10       0.44        0.03        0.10        0.44
1      50       0.05       0.18       0.22        0.08        0.28        0.66
2      60       0.15       0.40       0.16        0.23        0.68        0.82
3      70       0.20       0.20       0.12        0.43        0.88        0.94
4      80       0.35       0.08       0.06        0.78        0.96        1.00
5      90       0.15       0.04       0.00        0.93        1.00        1.00
6     100       0.07       0.00       0.00        1.00        1.00        1.00


In [10]:
random_numbers = []
if(input('Would you like to enter custom Random Numbers? (T/F) ') == 'T'):
    
    print(f'Enter {ndays*2} numbers between 1 and 100:\n')
    for i in range(0, ndays*2):
        n = input()
        random_numbers.append(int(n))
else:
    random_numbers = random.sample(range(1,100), ndays * 2)
    print(f'Generated numbers: {random_numbers}')

# scale numbers [1,100] ~ [0,1] to match cprob column
random_numbers = [round(n * 0.01, 2) for n in random_numbers]
print(f'\nScaled numbers: {random_numbers}')

Would you like to enter custom Random Numbers? (T/F) T
Enter 10 numbers between 1 and 100:

94
80
77
20
49
15
45
88
43
98

Scaled numbers: [0.94, 0.8, 0.77, 0.2, 0.49, 0.15, 0.45, 0.88, 0.43, 0.98]


## Call function to run simulation

In [11]:
simulation, profit = simulate_newspaper(cost_price, selling_price, scrap_price, order_quantity, ndays, newstype_dist, demand_dist, random_numbers)

display(simulation)
print(f'\nTotal Profit: {profit}')

,Day,RN Newsday Type,Newsday Type,RN Demand,Demand,Revenue,Lost Profit,Scrap Sale,Daily Profit
1,1,0.94,poor,0.80,60,30.0,0.0,0.5,7.4
2,2,0.77,fair,0.20,50,25.0,0.0,1.0,2.9
3,3,0.49,fair,0.15,50,25.0,0.0,1.0,2.9
4,4,0.45,fair,0.88,70,35.0,0.0,0.0,11.9
5,5,0.43,fair,0.98,90,35.0,3.4,0.0,8.5



Total Profit: 33.6
